In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os
import rasterio
from rasterio.mask import mask
# points = gpd.read_file('box_se.shp')
# nrows = 3
# ncols = 3
# out_name = "out_rasters/se/box_se_"

files = ["image_86_4BSW2R.tif.tif"]
#raster_file = "image_99_4BSW4R.tif.tif"
nrows = 2
ncols = 2

out_path = "out_rasters"

for f in files:
    with rasterio.open(f) as img:
        xmin, ymin, xmax, ymax = img.bounds

        width = xmax-xmin
        height = ymax-ymin

        wide = width / ncols
        length = height / nrows

        x = xmin
        y = ymin
        polygons = []
        for r in range(nrows):
            if r== nrows-1:
                ystep = length#+(height % nrows)
            else:
                ystep = length

            for c in range(ncols):        
                if (c % ncols) == 0:
                    x = xmin
                if c == ncols-1:
                    xstep = wide#+(width % ncols)
                else:
                    xstep = wide
                polygons.append(Polygon([(x,y), (x+xstep, y), (x+xstep, y+ystep), (x, y+ystep)]))
                x += xstep
            y += ystep


        grid = gpd.GeoDataFrame({'geometry':polygons})
        grid.crs = img.crs

        for i in range(len(grid)):
            bounds = grid.iloc[i]["geometry"]
            out_image, out_transform = mask(img, [bounds], crop=True)
            out_meta = img.meta
            out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
            out_file = os.path.join(out_path,f.split(".")[0]+"_"+str(i)+".tif")
            with rasterio.open(out_file, "w", **out_meta) as dest:
                dest.write(out_image)
        


# Old

In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os

# points = gpd.read_file('box_se.shp')
# nrows = 3
# ncols = 3
# out_name = "out_rasters/se/box_se_"

points = gpd.read_file('box_nw.shp')
nrows = 8
ncols = 8
labels_file = "labels_V5.shp"
out_name = "out_rasters/nw/box_nw_"

xmin, ymin, xmax, ymax = points.total_bounds

width = xmax-xmin
height = ymax-ymin

wide = width / ncols
length = height / nrows

x = xmin
y = ymin
polygons = []
for r in range(nrows):
    if r== nrows-1:
        ystep = length#+(height % nrows)
    else:
        ystep = length
    
    for c in range(ncols):        
        if (c % ncols) == 0:
            x = xmin
        if c == ncols-1:
            xstep = wide#+(width % ncols)
        else:
            xstep = wide
        polygons.append(Polygon([(x,y), (x+xstep, y), (x+xstep, y+ystep), (x, y+ystep)]))
        x += xstep
    y += ystep
        

grid = gpd.GeoDataFrame({'geometry':polygons})
grid.set_crs(epsg=4326, inplace=True)
grid.to_file("grid.shp")

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [2]:

for i in range(len(grid)):
    bounds = grid.iloc[i]["geometry"].bounds
    gdal_str = "gdal_rasterize -l " + labels_file.split(".")[0] + " -burn 1.0 -tr 2.306434877e-06 2.306434877e-06 -a_nodata 0.0 -te \
                " + str(bounds[0]) + " " + str(bounds[1]) + " "  + str(bounds[2]) + " " + str(bounds[3]) + " \
                " + "-ot Float32 -of GTiff " + labels_file + " " + out_name + str(i) + ".tif"
    os.system(gdal_str)
    
    

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...